In [9]:
# import
import os
import glob
import psycopg2
import datetime
import numpy as np
import pandas as pd

# 파일 불러오기
dir = 'C:\\oasis\\git\\data\\microdata\\'
glob.glob(dir + "*소규모상가.xlsx")

# set table name
table = 'm1.small_arcade'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)
    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

In [13]:
glob.glob(dir + "*소규모상가.xlsx")[0]

'C:\\oasis\\git\\data\\microdata\\2020_소규모상가.xlsx'

In [16]:
# 파일 불러와서 
for file in glob.glob(dir + "*소규모상가.xlsx"):
    
    df = pd.read_excel(file)
    
    # 컬럼명 변경
    df.columns = ['sample_gb', 'commercial_wide_area', 'cname', 'sido', 'own_type', 'own_cnt', 'land_use_nm1', 'land_use_nm2',
           'use_district_nm1', 'use_district_nm2', 'ground_floor', 'under_ground_floor', 'useapr_day', 'building_structure',
           'main_purps_nm', 'bc_rat', 'vl_rat', 'subway_distance', 'bus_distance', 'main_crossroad', 'road_interface_nm', 'indoor_parking_gb',
           'outdoor_parking_gb', 'passenger_elevator_gb ', 'freight_elevator_gb', 'plat_area', 'arch_area', 'net_leasable_area', 'common_area',
           'net_leasable_ratio', 'lease_contract_area', 'vacant_area', 'non_lease_area', 'special_area', 'etc_area', 'total_parking_area',
           'ground_parking_area', 'under_ground_parking_area', 'piriti_parking_area', 'outdoor_parking_area', 'jimok_nm', 'land_area', 'conversion_rate',
           'invest_floor', 'tenant_usage_status', 'business_type_large', 'business_type_small', 'net_leasable_ho_area', 'common_ho_area',
           'sum_area', 'contract_deposit_1st', 'contract_monthly_rent_sum_1st', 'contract_monthly_rent_1st',
           'selling_deposit_1st', 'selling_monthly_rent_sum_1st', 'selling_monthly_rent_1st', 'admin_cost_1st',
           'admin_actual_cost_1st', 'vacant_1', 'contract_deposit_2nd', 'contract_monthly_rent_sum_2nd',
           'contract_monthly_rent_2nd', 'selling_deposit_2nd', 'selling_monthly_rent_sum_2nd',
           'selling_monthly_rent_2nd', 'admin_cost_2nd', 'admin_actual_cost_2nd', 'vacant_2',
           'contract_deposit_3rd', 'contract_monthly_rent_sum_3rd', 'contract_monthly_rent_3rd',
           'selling_deposit_3rd', 'selling_monthly_rent_sum_3rd', 'selling_monthly_rent_3rd', 'admin_cost_3rd',
           'admin_actual_cost_3rd', 'vacant_3', 'current_contract_date', 'current_contract_term', 'first_current_contract_date', 'final_land_area_amt',
           'final_tot_area_amt']

    # base_ym 추가 
    df['base_ym'] = datetime.datetime.now().strftime("%Y%m")
    
    # 특수문자, nan 처리
    df = df.replace("%","", regex=True)
    df = df.replace({np.nan: None})

    # values 설정
    val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

    # insert
    for j in range(0, len(df), 10000):
        # DB Connect
        conn = psycopg2.connect(
            host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
            port=5439,
            dbname='dev',
            user='awsuser',
            password='cremaoAdmin1234qwer!!'
        )
        # 10000개 단위로 나누기
        tmp = df[j:j + 10000]

        # insert
        execute_mogrify(conn, tmp, table, val)
        print(datetime.datetime.now(), ' : ', j)
        tmp = pd.DataFrame()

    # row count
    print(f'Done : {file}')

2022-12-15 18:07:23.565119  :  0
Done : C:\oasis\git\data\microdata\2020_소규모상가.xlsx


ValueError: Length mismatch: Expected axis has 83 elements, new values have 82 elements

### test

In [51]:
df = df[['base_ym','sample_gb', 'commercial_wide_area', 'cname', 'sido', 'own_type', 'own_cnt', 'land_use_nm1', 'land_use_nm2',
       'use_district_nm1', 'use_district_nm2', 'ground_floor', 'under_ground_floor', 'useapr_day', 'building_structure',
       'main_purps_nm', 'bc_rat', 'vl_rat', 'subway_distance', 'bus_distance', 'main_crossroad', 'road_interface_nm', 'indoor_parking_gb',
       'outdoor_parking_gb', 'passenger_elevator_gb ', 'freight_elevator_gb', 'plat_area', 'arch_area', 'net_leasable_area', 'common_area',
       'net_leasable_ratio', 'lease_contract_area', 'vacant_area', 'non_lease_area', 'special_area', 'etc_area', 'total_parking_area',
       'ground_parking_area', 'under_ground_parking_area', 'piriti_parking_area', 'outdoor_parking_area', 'jimok_nm', 'land_area', 'conversion_rate',
       'invest_floor', 'tenant_usage_status', 'business_type_large', 'business_type_small', 'net_leasable_ho_area', 'common_ho_area',
       'sum_area', 'contract_deposit_1st', 'contract_monthly_rent_sum_1st', 'contract_monthly_rent_1st',
       'selling_deposit_1st', 'selling_monthly_rent_sum_1st', 'selling_monthly_rent_1st', 'admin_cost_1st',
       'admin_actual_cost_1st', 'vacant_1', 'contract_deposit_2nd', 'contract_monthly_rent_sum_2nd',
       'contract_monthly_rent_2nd', 'selling_deposit_2nd', 'selling_monthly_rent_sum_2nd',
       'selling_monthly_rent_2nd', 'admin_cost_2nd', 'admin_actual_cost_2nd', 'vacant_2',
       'contract_deposit_3rd', 'contract_monthly_rent_sum_3rd', 'contract_monthly_rent_3rd',
       'selling_deposit_3rd', 'selling_monthly_rent_sum_3rd', 'selling_monthly_rent_3rd', 'admin_cost_3rd',
       'admin_actual_cost_3rd', 'vacant_3', 'current_contract_date', 'current_contract_term', 'first_current_contract_date', 'final_land_area_amt',
       'final_tot_area_amt']]